In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset
import torch.optim as optim
import torch.nn.functional as F
import string
!pip install clean-text
from cleantext import clean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=bdcf4461f66a47777cff615db6ef84675de3caae5bdb1796f80500eb01be8d7f
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.11.0
    Uninstalling emoji-2.11.0:
      Successfully uninstalled emoji-2.11.0


In [2]:
# Load the dataset from Hugging Face
dataset = load_dataset("123rc/medical_text")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
train_data = dataset["train"]
test_data = dataset["test"]

In [4]:
# Convert train and test splits to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [5]:
# Rename two columns 
train_df.rename(columns={'condition_label': 'label', 'medical_abstract': 'text'}, inplace=True)

In [6]:
# Rename two columns 
test_df.rename(columns={'condition_label': 'label', 'medical_abstract': 'text'}, inplace=True)

In [7]:
train_df['label'] = train_df['label'] - 1
test_df['label'] = test_df['label'] - 1

In [8]:
train_df['label'].value_counts()

label
4    3844
0    2530
3    2441
2    1540
1    1195
Name: count, dtype: int64

In [9]:
train_df['label'].nunique()

5

In [10]:
test_df.head()

,label,text
0,2,Obstructive sleep apnea following topical orop...
1,4,Neutrophil function and pyogenic infections in...
2,4,A phase II study of combined methotrexate and ...
3,0,Flow cytometric DNA analysis of parathyroid tu...
4,3,Paraneoplastic vasculitic neuropathy: a treata...


In [11]:
train_df.shape

(11550, 2)

In [12]:
test_df.shape

(2888, 2)

In [13]:
# Removing Repeated Punctuations
def remove_repeated_punctuation(text):
    punctuations = set(string.punctuation)
    cleaned_text = []
    for char in text:
        if char in punctuations:
            punctuations.remove(char)
            cleaned_text.append(char)
        elif char not in punctuations:
            punctuations = set(string.punctuation)
            cleaned_text.append(char)
    return ''.join(cleaned_text)

# Apply the remove_repeated_punctuation function to the 'review' column
train_df['text'] = train_df['text'].apply(remove_repeated_punctuation)

train_df.head()

,label,text
0,4,Tissue changes around loose prostheses. A cani...
1,0,Neuropeptide Y and neuron-specific enolase lev...
2,1,"Sexually transmitted diseases of the colon, re..."
3,0,Lipolytic factors associated with murine and h...
4,2,Does carotid restenosis predict an increased r...


In [14]:
test_df['text'] = test_df['text'].apply(remove_repeated_punctuation)

test_df.head()

,label,text
0,2,Obstructive sleep apnea following topical orop...
1,4,Neutrophil function and pyogenic infections in...
2,4,A phase II study of combined methotrexate and ...
3,0,Flow cytometric DNA analysis of parathyroid tu...
4,3,Paraneoplastic vasculitic neuropathy: a treata...


In [15]:
# NLTK library to remove Stopwords.
from nltk.corpus import stopwords

In [16]:
stopword = stopwords.words('english')

In [17]:
# Store the length of each review before removing less important words
train_df['length_before'] = train_df['text'].apply(len)

In [18]:
# Function
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopword:
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

# Calling Function 
train_df['text'] = train_df['text'].apply(remove_stopwords)

train_df.head()

,label,text,length_before
0,4,Tissue changes around loose prostheses. A cani...,1057
1,0,Neuropeptide Y neuron-specific enolase levels...,1207
2,1,"Sexually transmitted diseases colon, rectum,...",1771
3,0,Lipolytic factors associated murine human ca...,1006
4,2,Does carotid restenosis predict increased ris...,1489


In [19]:
# Store the length of each review before removing less important words
train_df['length_after'] = train_df['text'].apply(len)
train_df.head()

,label,text,length_before,length_after
0,4,Tissue changes around loose prostheses. A cani...,1057,898
1,0,Neuropeptide Y neuron-specific enolase levels...,1207,1008
2,1,"Sexually transmitted diseases colon, rectum,...",1771,1504
3,0,Lipolytic factors associated murine human ca...,1006,878
4,2,Does carotid restenosis predict increased ris...,1489,1287


In [20]:
# Calling Function 
test_df['text'] = test_df['text'].apply(remove_stopwords)

test_df.head()

,label,text
0,2,Obstructive sleep apnea following topical orop...
1,4,Neutrophil function pyogenic infections bone...
2,4,A phase II study combined methotrexate tenip...
3,0,Flow cytometric DNA analysis parathyroid tumo...
4,3,Paraneoplastic vasculitic neuropathy: treatab...


In [21]:
# Apply text cleaning function
train_df['text'] = train_df['text'].apply(lambda x: clean(x,
                                                        fix_unicode=True,
                                                        to_ascii=True,
                                                        lower=True,
                                                        no_line_breaks=False,
                                                        no_urls=True,
                                                        no_emails=True,
                                                        no_phone_numbers=True,
                                                        no_numbers=True,
                                                        no_currency_symbols=True,
                                                        no_punct=True,
                                                        replace_with_punct="",
                                                        replace_with_url="<URL>",
                                                        replace_with_email="<EMAIL>",
                                                        replace_with_phone_number="<PHONE>",
                                                        replace_with_number="<NUMBER>",
                                                        replace_with_currency_symbol="<CUR>",
                                                        lang="en"
                                                        ))

train_df.head()

,label,text,length_before,length_after
0,4,tissue changes around loose prostheses a canin...,1057,898
1,0,neuropeptide y neuronspecific enolase levels b...,1207,1008
2,1,sexually transmitted diseases colon rectum anu...,1771,1504
3,0,lipolytic factors associated murine human canc...,1006,878
4,2,does carotid restenosis predict increased risk...,1489,1287


In [22]:
# Apply text cleaning function
test_df['text'] = test_df['text'].apply(lambda x: clean(x,
                                                        fix_unicode=True,
                                                        to_ascii=True,
                                                        lower=True,
                                                        no_line_breaks=False,
                                                        no_urls=True,
                                                        no_emails=True,
                                                        no_phone_numbers=True,
                                                        no_numbers=True,
                                                        no_currency_symbols=True,
                                                        no_punct=True,
                                                        replace_with_punct="",
                                                        replace_with_url="<URL>",
                                                        replace_with_email="<EMAIL>",
                                                        replace_with_phone_number="<PHONE>",
                                                        replace_with_number="<NUMBER>",
                                                        replace_with_currency_symbol="<CUR>",
                                                        lang="en"
                                                        ))

test_df.head()

,label,text
0,2,obstructive sleep apnea following topical orop...
1,4,neutrophil function pyogenic infections bone m...
2,4,a phase ii study combined methotrexate tenipos...
3,0,flow cytometric dna analysis parathyroid tumor...
4,3,paraneoplastic vasculitic neuropathy treatable...


In [23]:
texts = train_df['text'].tolist()
labels = train_df['label'].tolist()

In [24]:
train_texts, val_texts, train_labels, val_labelss = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [25]:
test_texts = test_df['text'].tolist()
test_labels = test_df['label'].tolist()

In [26]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = [str(text) for text in texts]
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [27]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [28]:
# Set up parameters
bert_model_name = 'bert-base-uncased'
num_classes = 5
max_length = 512
batch_size = 8

In [29]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labelss, tokenizer, max_length)
test_dataset = TextClassificationDataset(test_texts, test_labels, tokenizer, max_length)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERTClassifier(bert_model_name, num_classes).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [31]:
epochs = 20
best_roc_auc = 0.0
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 3
gradient_accumulation_steps = 10

# Set the optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_dataloader) * epochs // gradient_accumulation_steps
)


In [32]:
# Training
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
            optimizer.step()
            scheduler.step()
            
    model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            
    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_dataloader)
    train_loss /= len(train_dataloader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')

    print(f'Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1: {f1}, Micro F1: {micro_f1:.4f}, Macro Roc Auc: {macro_roc_auc:.4f}')
            
   # Implement early stopping
    if epoch > 0 and macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0


Epoch: 1/20, Training Loss: 1.5584, Validation Loss: 1.4521
Accuracy: 0.3861, Recall: 0.3861, Precision: 0.3872, F1: 0.2975288308615443, Micro F1: 0.3861, Macro Roc Auc: 0.6506


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 2/20, Training Loss: 1.1733, Validation Loss: 0.9843
Accuracy: 0.5983, Recall: 0.5983, Precision: 0.5915, F1: 0.5884366711241099, Micro F1: 0.5983, Macro Roc Auc: 0.8633
Epoch: 3/20, Training Loss: 0.9169, Validation Loss: 0.9111
Accuracy: 0.6277, Recall: 0.6277, Precision: 0.6338, F1: 0.6043868576861493, Micro F1: 0.6277, Macro Roc Auc: 0.8830
Epoch: 4/20, Training Loss: 0.8292, Validation Loss: 0.8877
Accuracy: 0.6234, Recall: 0.6234, Precision: 0.6246, F1: 0.6073468924639389, Micro F1: 0.6234, Macro Roc Auc: 0.8923
Epoch: 5/20, Training Loss: 0.7703, Validation Loss: 0.8678
Accuracy: 0.6255, Recall: 0.6255, Precision: 0.6201, F1: 0.6150170411327355, Micro F1: 0.6255, Macro Roc Auc: 0.8933
Epoch: 6/20, Training Loss: 0.7130, Validation Loss: 0.8812
Accuracy: 0.6152, Recall: 0.6152, Precision: 0.6129, F1: 0.6021190003553804, Micro F1: 0.6152, Macro Roc Auc: 0.8901
EarlyStopping counter: 1 out of 3
Epoch: 7/20, Training Loss: 0.6773, Validation Loss: 0.9207
Accuracy: 0.6177, Rec

In [33]:
model.eval()

test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())


In [34]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

test_preds_class = np.argmax(test_preds, axis=1)

report = classification_report(test_labels, test_preds_class, digits = 4)

print(report)

              precision    recall  f1-score   support

           0     0.6696    0.8389    0.7447       633
           1     0.5389    0.6254    0.5789       299
           2     0.5644    0.5922    0.5779       385
           3     0.6898    0.7180    0.7036       610
           4     0.5599    0.4131    0.4754       961

    accuracy                         0.6167      2888
   macro avg     0.6045    0.6375    0.6161      2888
weighted avg     0.6098    0.6167    0.6070      2888

